In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.svm import SVC
import sys
import scipy.io
import os
import openpyxl
import re
import pickle
import time
import scipy.io as sio

In [ ]:
import keras
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
# from keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping
from keras.models import Sequential, Model
from keras import models
from keras.layers import Flatten, Dense, Concatenate, Reshape, LSTM,Conv1D,MaxPooling1D, concatenate
from keras import backend as K
from keras.layers import Layer,Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D

In [ ]:
num_classes = 3
channels=62
drive='E'

In [ ]:
mypath='feature matrix address'
file = os.listdir(mypath)
file

In [ ]:
label_file='label address'
label_fname = os.path.join(mypath,label_file)
label = np.load(label_fname,allow_pickle='TRUE')
label.shape

In [ ]:
def data_1Dto2D(data, Y=8, X=9):
    data_2D = np.zeros([Y, X])
    data_2D[0] = (0, 0, data[3], data[0], data[1], data[2], data[4], 0,0 )
    data_2D[1] = (data[5], data[6], data[7], data[8], data[9], data[10], data[11], data[12], data[13])
    data_2D[2] = (data[14], data[15], data[16], data[17], data[18], data[19], data[20], data[21], data[22])
    data_2D[3] = (data[23], data[24], data[25], data[26], data[27], data[28], data[29], data[30], data[31])
    data_2D[4] = (data[32], data[33], data[34], data[35],data[36], data[37], data[38], data[39], data[40])
    data_2D[5] = (data[41], data[42], data[43], data[44], data[45], data[46], data[47], data[48], data[49])
    data_2D[6] = (0, data[50], data[51], data[52], data[53], data[54], data[55], data[56], 0)
    data_2D[7] = (0, 0, data[57], data[58], data[59], data[60], data[61], 0, 0)
    # return shape:8*9
    return data_2D

In [ ]:
def image_5_d(data11):
    indexx=[0,1,2,3,4]
    # lst=[0,1,7,8,54,62,63,64,70,71]
    def eeg_image_wave(df,index):
#         print(index)
        wave_index=[index+i*5 for i in range(62)]
#         print(wave_index)
        data_1D=df[:,wave_index]
#         print(data_1D)
        data_3D=np.empty([0,8,9])
        for i in range(data_1D.shape[0]):
            data_2D_temp=data_1Dto2D(data_1D[i], Y=8, X=9)
            data_2D_temp=data_2D_temp.reshape(1, 8, 9)
            data_3D=np.vstack([data_3D, data_2D_temp])
#         print(data_3D.shape)
        return data_3D
    delta_wave_image=eeg_image_wave(data11,indexx[0])
    theta_wave_image=eeg_image_wave(data11,indexx[1])
    alpha_wave_image=eeg_image_wave(data11,indexx[2])
    beta_wave_data=eeg_image_wave(data11,indexx[3])
    gamma_wave_data=eeg_image_wave(data11,indexx[4])
    image_5_d1=np.stack((delta_wave_image,theta_wave_image,alpha_wave_image,beta_wave_data
               ,gamma_wave_data),axis=1)
    print(f'shape of the eeg image data: {image_5_d1.shape}')
    return image_5_d1

In [ ]:
def cnn_model(X_train_cnn):
    x_in=keras.Input(shape=(X_train_cnn.shape[1:]))
    x=Conv2D(256, 5, activation='relu', padding='same', name='conv1')(x_in)
    x=Conv2D(256, 5, activation='relu', padding='same', name='conv2')(x)
    x=MaxPooling2D(2, 2, name='pool2')(x)
    x=Conv2D(128, 4, activation='relu', padding='same', name='conv3')(x)
    x=MaxPooling2D(2, 2, name='pool3')(x)####AveragePooling2D
    x=Flatten(name='fla1')(x)
    x=Dense(64, activation='relu', name='dense1')(x)
    x=Dense(3, activation='softmax', name='out')(x)
    model1=keras.Model(inputs=x_in, outputs=x)   
    return model1

In [ ]:
# for i in range(len(file)):
from sklearn import metrics
acc_list = []
std_list = []
f1_score_acc_list = []
f1_score_std_list = []
all_acc = []
all_f1_score = []
scaling='no'
num_classes=3
folds=5
# for i in range(1):
for i in range(len(file)):
    data_fname = os.path.join(mypath,file[i])
    data = np.load(data_fname,allow_pickle='TRUE')
    data1=data.reshape(data.shape[0],data.shape[1]*data.shape[2])
    ############
    if scaling== 'yes':
        data_dnn = preprocessing.scale(data1, axis=0, with_mean=True, with_std=True, copy=True)
        print('scaling is performed')
    else:
        data_dnn=data1
        print('scaling is not performed')
    ##############################
    y_labels = label
    y_labels = to_categorical(y_labels, 3)
############################
####################################
    data_cnn= image_5_d(data_dnn)
    data_cnn= data_cnn.transpose([0, 2, 3, 1])
################################
    print(f'data_cnn shape is:{data_cnn.shape}')
    kfold = StratifiedKFold(n_splits=folds, shuffle=True, random_state=5)
    # cvscores = []
    # create model
    for train, test in kfold.split(data_cnn, y_labels.argmax(1)):
#         print(train)
#         print(test)
        X_train_cnn = data_cnn[train]
        y_train = y_labels[train]
        X_test_cnn = data_cnn[test]
        y_test = y_labels[test]
        #############################
        model=cnn_model(X_train_cnn)
        model.compile(loss=keras.losses.categorical_crossentropy,optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])
        model.fit(x=X_train_cnn, y=y_train, epochs=100,batch_size=128,verbose=0)
        scores = model.evaluate(X_test_cnn, y_test, verbose=0)
       #############
        y_pred=model.predict(X_test_cnn)
        y_true=y_test
        y_pred=np.argmax(y_pred, axis=1)
        y_true=np.argmax(y_true, axis=1)
        f1_score=metrics.f1_score(y_true, y_pred, average='macro')
##################
        all_acc.append(scores[1] * 100)
        all_f1_score.append(f1_score)
        #############
    acc_list.append(np.mean(all_acc))
    std_list.append(np.std(all_acc))
    f1_score_acc_list.append(np.mean(all_f1_score))
    f1_score_std_list.append(np.std(all_f1_score))

In [ ]:
model.summary()

In [ ]:
results=pd.DataFrame()
results['Acc_all']=acc_list
results['Std_all']=std_list
results['f1_score_acc']=f1_score_acc_list
results['f1_score_std']=f1_score_std_list
results